# MALLORN - train an Echo State Network
Comment 31/01/26: The key outcome of this notebook is adding the "ESN" feature to the updated train/test logs

In [1]:
# Imports
import numpy as np
import pandas as pd

In [2]:
def compute_filter_stats(df):
    vals = {f: [] for f in filters}

    for f in filters:
        v = df.loc[df["Filter"] == f, "Fluxcorr"].values
        vals[f].extend(v)

    stats = {}
    for f in filters:
        arr = np.asarray(vals[f])
        stats[f] = (np.nanmean(arr), np.nanstd(arr) + 1e-6)

    return stats


In [3]:
def states_to_features(states, washout=10):
    s = states[washout:]
    return np.hstack([np.nanmean(s,axis=0), np.nanstd(s,axis=0)])

In [4]:
def compute_dt_scale(df):
    dts = []
    t = df["time"].values
    if len(t) > 1:
        dts.extend(np.diff(np.sort(t)))
    return np.nanmedian(dts)


In [5]:
def build_event_sequence(df,peak,z):
    df=df.sort_values('Time (MJD)')
    times=df['time'].values
    values=df['Fluxcorr'].values
    filters=df['Filter'].values
    seq=[]
    last_t = df["time"].iloc[0]

    for _, row in df.iterrows():
        v = row["Fluxcorr"]
        f = row["Filter"]
        t = row["time"]

        dt = t - last_t
        last_t = t

        onehot = np.zeros(6)        # ← FIXED SIZE
        onehot[f2idx[f]] = 1.0

        x = np.concatenate([[v, dt], onehot])  # length = 8
        seq.append(x)

    seq = np.asarray(seq)

    assert seq.shape[1] == 8, f"Got {seq.shape[1]} features!"

    return seq 

In [6]:
from reservoirpy.nodes import Reservoir
from reservoirpy.mat_gen import sparse

reservoir = Reservoir(
    units=1500,
    sr=0.9,
    lr=0.2,
    input_scaling=0.5,
    W=sparse.random(1500, 1500, density=0.05),
    seed=42
)

In [139]:
from sklearn.linear_model import LogisticRegression
from scipy.sparse.linalg import eigs
filters=['u','g','r','i','z','y']
f2idx={f:i for i,f in enumerate(filters)}
df_log=pd.read_csv('content/mallorn-astronomical-classification-challenge/train_log_updated.csv')
# obj ids
objects=df_log.object_id.unique()
X,y=[],[]
for obj in objects:
    split=df_log.split[df_log.object_id==obj].values[0]
    peak=df_log.peak_time_i[df_log.object_id==obj].values[0]
    label=df_log.target[df_log.object_id==obj].values[0]
    z=df_log.Z[df_log.object_id==obj].values[0]
    df_lc=pd.read_csv('content/mallorn-astronomical-classification-challenge/{}/train_full_lightcurves.csv'.format(split))
    df=df_lc[df_lc.object_id==obj].copy()
    df=df[df.Flux/df.Flux_err>=3]
    #if len(df) < 5:   # choose a reasonable threshold
     #   continue
    df['time']=(df['Time (MJD)']-peak)/(1+z)

    dt_scale = compute_dt_scale(df)

    f_stats=compute_filter_stats(df)

    W=sparse.random(1500, 1500, density=0.05)
    #rho = max(abs(eigs(W, k=1, return_eigenvectors=False)))
    #W *= sr / rho
    
    seq=build_event_sequence(df,peak,z)
    reservoir = Reservoir(
    units=1500,
    sr=0.7,
    lr=0.2,
    input_scaling=0.5,
    W=W,
    seed=42
)
    states=reservoir.run(seq)
    feat=states_to_features(states)

    if not np.isfinite(feat).all():
        feat = np.nan_to_num(feat, nan=0.0, posinf=0.0, neginf=0.0)
    X.append(feat)
    y.append(label)  
    
    

In [144]:
# Feed to classifier
clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",random_state=10
)
clf.fit(X, y) 
from sklearn.model_selection import StratifiedKFold, cross_val_score

#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#scores = cross_val_score(clf, X, y, cv=skf, scoring='f1')
#print(scores.mean(), scores.std())



In [145]:
from sklearn.model_selection import TunedThresholdClassifierCV
tuned_model=TunedThresholdClassifierCV(clf,scoring='f1',cv=5).fit(X,y)
print(f"Cut-off at {tuned_model.best_threshold_:.3f}")
print('best score: {:.3f}'.format(tuned_model.best_score_))

Cut-off at 0.451
best score: 0.144


In [159]:
# Get OOF probability for training data
from sklearn.model_selection import StratifiedKFold

X = np.array(X)   # shape: (n_samples, n_features)
y = np.array(y)

n_samples = len(X)
n_classes = len(np.unique(y))

oof_proba = np.zeros((n_samples, n_classes))

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

for train_idx, val_idx in skf.split(X, y):
    X_train = X[train_idx]
    X_val   = X[val_idx]
    y_train = y[train_idx]

    clf = LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        random_state=10
    )

    clf.fit(X_train, y_train)
    oof_proba[val_idx] = clf.predict_proba(X_val)


In [163]:
# Add to train log
df_log['ESN']=oof_proba[:,1]
df_log.to_csv('content/mallorn-astronomical-classification-challenge/train_log_updated.csv')

In [146]:
# Test data
from sklearn.linear_model import LogisticRegression
from scipy.sparse.linalg import eigs
filters=['u','g','r','i','z','y']
f2idx={f:i for i,f in enumerate(filters)}
df_test=pd.read_csv('content/mallorn-astronomical-classification-challenge/test_log_updated.csv')
# obj ids
objects=df_test.object_id.unique()
Xtest=[]
for obj in objects:
    #obj=objects[17]
    split=df_test.split[df_test.object_id==obj].values[0]
    peak=df_test.peak_time_i[df_test.object_id==obj].values[0]
    #label=df_test.target[df_test.object_id==obj].values[0]
    z=df_test.Z[df_test.object_id==obj].values[0]
    df_lc=pd.read_csv('content/mallorn-astronomical-classification-challenge/{}/test_full_lightcurves.csv'.format(split))
    df=df_lc[df_lc.object_id==obj].copy()
    df=df[df.Flux/df.Flux_err>=3]
    #if len(df) < 5:   # choose a reasonable threshold
     #   continue
    df['time']=(df['Time (MJD)']-peak)/(1+z)

    dt_scale = compute_dt_scale(df)

    f_stats=compute_filter_stats(df)

    W=sparse.random(1500, 1500, density=0.05)
    #rho = max(abs(eigs(W, k=1, return_eigenvectors=False)))
    #W *= sr / rho
    
    seq=build_event_sequence(df,peak,z)
    reservoir = Reservoir(
    units=1500,
    sr=0.7,
    lr=0.2,
    input_scaling=0.5,
    W=W,
    seed=42
)
    states=reservoir.run(seq)
    feat=states_to_features(states)

    if not np.isfinite(feat).all():
        feat = np.nan_to_num(feat, nan=0.0, posinf=0.0, neginf=0.0)
    Xtest.append(feat)
    #y.append(label)  
    
    

## Final important cell

In [150]:
# Add to test log
yproba=tuned_model.predict_proba(Xtest)
df_test['ESN']=yproba[:,1]
df_test.to_csv('content/mallorn-astronomical-classification-challenge/test_log_updated.csv')

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])
from sklearn.model_selection import GridSearchCV

param_grid = {
    'clf__C': [0.0005,0.0008,0.001,0.003,0.005,0.01],
    'clf__penalty': ['l2'],
}

grid = GridSearchCV(
    pipe,
    param_grid,
    scoring='f1',
    cv=skf,
    n_jobs=-1
)

grid.fit(X, y)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/opt/anaconda3/lib/python3.13/sit

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.0005, 0.0008, 0.001, 0.003, 0.005, 0.01],
                         'clf__penalty': ['l2']},
             scoring='f1')

In [34]:
print(grid.best_params_, grid.best_score_)


{'clf__C': 0.001, 'clf__penalty': 'l2'} 0.13477401050453403


In [ ]:
from sklearn.decomposition import PCA

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # keep 95% variance
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])


## Hyperparameter sweep

In [66]:
# Precompute sequences and labels once
sequences = []
labels = []

for obj in objects:
    split = df_log.split[df_log.object_id == obj].values[0]
    peak  = df_log.peak_time_i[df_log.object_id == obj].values[0]
    label = df_log.target[df_log.object_id == obj].values[0]
    z     = df_log.Z[df_log.object_id == obj].values[0]

    df_lc = pd.read_csv(
        f'content/mallorn-astronomical-classification-challenge/{split}/train_full_lightcurves.csv'
    )
    df = df_lc[df_lc.object_id == obj].copy()
    df = df[df.Flux / df.Flux_err >= 3]

    if len(df) < 5:
        continue

    df['time'] = (df['Time (MJD)'] - peak) / (1 + z)

    seq = build_event_sequence(df, peak, z)

    sequences.append(seq)
    labels.append(label)

y = np.array(labels)


In [67]:
# get esn features for specific reservoir
def extract_esn_features(sequences, reservoir):
    X = []

    for seq in sequences:
        states = reservoir.run(seq)
        feat = states_to_features(states)

        if not np.isfinite(feat).all():
            feat = np.nan_to_num(feat)

        X.append(feat)

    return np.asarray(X)


In [68]:
# cv f1 evaluation function
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def evaluate_reservoir(X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    f1s = []

    for train_idx, test_idx in skf.split(X, y):
        pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('clf', LogisticRegression(
                max_iter=1000,
                class_weight='balanced',
                C=0.001
            ))
        ])

        pipe.fit(X[train_idx], y[train_idx])
        y_pred = pipe.predict(X[test_idx])
        f1s.append(f1_score(y[test_idx], y_pred))

    return np.mean(f1s), np.std(f1s)


In [79]:
import warnings
warnings.filterwarnings('ignore')
from scipy import sparse
import itertools

X_prev=None

param_grid = {
    'units': [1000, 1500],
    'sr': [0.7, 0.9],#, 1.1],
    'lr': [0.1, 0.2],#, 0.4],
    'input_scaling': [0.2, 0.5]
}

results = []

for units, sr, lr, input_scaling in itertools.product(
        param_grid['units'],
        param_grid['sr'],
        param_grid['lr'],
        param_grid['input_scaling']):

    print(f"Evaluating: units={units}, sr={sr}, lr={lr}, input_scaling={input_scaling}")

    W = sparse.random(
        units, units,
        density=0.05,
        random_state=42,
        data_rvs=np.random.randn
    )

    reservoir = Reservoir(
        units=units,
        sr=sr,
        lr=lr,
        input_scaling=input_scaling,
        W=W,
        seed=1
    )

    X = extract_esn_features(sequences, reservoir)
    print("X mean:", X.mean())
    print("X std :", X.std())
    print("X hash:", np.round(X, 4).sum())

   # if X_prev is not None:
    #    diff = np.mean(np.abs(X - X_prev))
     #   print("ΔX from previous config:", diff)
    #X_prev = X.copy()


    #print("Feature std (global):", np.std(X))
    #print("Feature std (per-dim, mean):", np.mean(np.std(X, axis=0)))

    mean_f1, std_f1 = evaluate_reservoir(X, y)

    results.append({
        'units': units,
        'sr': sr,
        'lr': lr,
        'input_scaling': input_scaling,
        'f1_mean': mean_f1,
        'f1_std': std_f1
    })

    print(f" → F1 = {mean_f1:.3f} ± {std_f1:.3f}")


Evaluating: units=1000, sr=0.7, lr=0.1, input_scaling=0.2
X mean: 1.4695107397656404e-05
X std : 0.006153394556280123
X hash: 89.4354
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.7, lr=0.1, input_scaling=0.5
X mean: 1.201660116538286e-05
X std : 0.006517420990262117
X hash: 73.1327
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.7, lr=0.2, input_scaling=0.2
X mean: 1.9154170845843023e-05
X std : 0.00915786935729873
X hash: 116.5711
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.7, lr=0.2, input_scaling=0.5
X mean: 2.1959292169792714e-05
X std : 0.009509626190231876
X hash: 133.64299999999997
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.9, lr=0.1, input_scaling=0.2
X mean: 1.7261350301094355e-05
X std : 0.0061817166184641795
X hash: 105.0526
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.9, lr=0.1, input_scaling=0.5
X mean: 9.561759678318645e-06
X std : 0.006691762871523607
X hash: 58.191199999999995
 → F1 = 0.074 ± 0.037
Evaluating: units=1000, sr=0.9, lr=0.2, in

In [84]:
states = reservoir.run(sequences[10])

print("states mean abs:", np.nanmean(np.abs(states)))
print("states std:", states.std())
print("per-unit std (mean):", np.mean(states.std(axis=0)))
print("last state std:", states[-1].std())


states mean abs: nan
states std: nan
per-unit std (mean): nan
last state std: nan


In [78]:
seq = sequences[110]

print("seq shape:", seq.shape)
print("seq finite:", np.isfinite(seq).all())
print("seq min/max:", np.nanmin(seq), np.nanmax(seq))


seq shape: (55, 8)
seq finite: True
seq min/max: 0.0 232.6805004314095


In [81]:
res1 = Reservoir(units=1500, sr=0.5, lr=0.01, input_scaling=0.1, W=W, seed=0)
res2 = Reservoir(units=1500, sr=2.0, lr=0.5,  input_scaling=5.0, W=W, seed=0)

X1 = extract_esn_features(sequences, res1)
X2 = extract_esn_features(sequences, res2)

print("Mean |ΔX|:", np.mean(np.abs(X1 - X2)))
print("Cosine similarity:", 
      np.dot(X1.ravel(), X2.ravel()) /
      (np.linalg.norm(X1.ravel()) * np.linalg.norm(X2.ravel())))


Mean |ΔX|: 0.00015160165033321264
Cosine similarity: 0.5299330848371515


In [100]:
# crank nonlinearity + temporal bins
reservoir = Reservoir(
    units=1500,
    sr=1.5,
    lr=0.05,
    input_scaling=5.0,
    W=W,
    seed=0
)

X = extract_esn_features(sequences, reservoir)
print("X std:", X.std())

mean_f1, _ = evaluate_reservoir(X, y)
print("F1:", mean_f1)


X std: 0.004702357574053009


TypeError: only integer scalar arrays can be converted to a scalar index